In [17]:
import os
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.agents import LlmAgent
help(LlmAgent)

print("OS & ADK components imported successfully.")

Help on class LlmAgent in module google.adk.agents.llm_agent:

class LlmAgent(google.adk.agents.base_agent.BaseAgent)
 |  LlmAgent(*, name: str, description: str = '', parent_agent: Optional[google.adk.agents.base_agent.BaseAgent] = None, sub_agents: list[google.adk.agents.base_agent.BaseAgent] = <factory>, before_agent_callback: Union[Callable[[google.adk.agents.callback_context.CallbackContext], Union[Awaitable[Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]], list[Callable[[google.adk.agents.callback_context.CallbackContext], Union[Awaitable[Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]]], NoneType] = None, after_agent_callback: Union[Callable[[google.adk.agents.callback_context.CallbackContext], Union[Awaitable[Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]], list[Callable[[google.adk.agents.callback_context.CallbackContext], Union[Awaitable[Optional[google.genai.types.Content]], google.genai.

In [2]:
try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Gemini API key setup complete.")
except Exception as e:
  print(f"Failed: {e}")

Gemini API key setup complete.


### Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [3]:
retry_config = types.HttpRetryOptions(
  attempts=5, #Maximum retry attempts
  exp_base=7, #Delay multiplier
  initial_delay=1,
  http_status_codes=[429,500,503,504], #Retry on these HTTP errors.

)

### Building Custom Function Tools

#### Currency Converter Agent

This agent can convert currency from one denomination to antother and calculates the feed to do the conversion. The agent has two custom tools and follows the workflow:
1. Fee Lookup Tool - Finds transaction fees for the conversion
2. Exchange Rate Toll - Gets currency conversion rates
3. Calculation Step - Calculates the total conversion cost including the fees

In [9]:
def get_fee_for_payment_method(method: str)-> dict:
  """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """

  #This simulates looking up a company's internal fee structure.
  fee_database = {
    "platinum credit card":0.02, #2%
    "gold debit card":0.035, #3.5%,
    "bank transfer":0.01 #1%
  }

  fee = fee_database.get(method.lower())
  if fee is not None:
    return {"status":"success","fee_percentage":fee}
  else:
    return{
      "status":"error",
      "error_message": f"Payment method '{method}' not found",
    }

print ("Fee Look up function created.")
print(f"Test: {get_fee_for_payment_method('platinum credit card')}")

Fee Look up function created.
Test: {'status': 'success', 'fee_percentage': 0.02}


In [10]:
def get_exchange_rate(base_currency: str, target_currency: str)->dict:
  """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

  #static data simulating a live exchange rate API
  #In production this would call sth like : requests.get("api.exchangerates.com")

  rate_database = {
    "usd":{
      "eur":0.93, #euro
      "jpy":157.5, #yen
      "inr":93.58, #rupee
    }
  }

  # input validation and processing

  base = base_currency.lower()
  target = target_currency.lower()

  #return structured result with status

  rate = rate_database.get(base, {}).get(target)
  if rate is not None:
    return{"status":"success","rate":rate}
  else:
    return{
      "status":"Error",
      "error_message":f"Unsuported currency pair {base_currency}/{target_currency}"}

print("Exchange rate function created.")
print(f"Test: {get_exchange_rate('usd','eur')}")
print(f"Test2: {get_exchange_rate('eur','yen')}")


Exchange rate function created.
Test: {'status': 'success', 'rate': 0.93}
Test2: {'status': 'Error', 'error_message': 'Unsuported currency pair eur/yen'}


In [12]:
#Currency agent with custom function tools

currency_agent = LlmAgent(
  name = "currency_agent",
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_exchange_rate,get_fee_for_payment_method]
)
print("currency_agent created with custom function tools.")
print("Available tools:"+"\n"+"- get_fee_for_payment_method : Looks up company fee structure."+"\n"+"- get_exchange_rate : Gets current exchange rates.")


currency_agent created with custom function tools.
Available tools:
- get_fee_for_payment_method : Looks up company fee structure.
- get_exchange_rate : Gets current exchange rates.


In [18]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
response = await currency_runner.run_debug("I want to conver 5000 us dollars to euros using my platinum credit card. How much will i receive?")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > I want to conver 5000 us dollars to euros using my platinum credit card. How much will i receive?


currency_agent > You will receive 4557 EUR.

Here's how that's calculated:
1. A fee of 2.0% is charged for using a platinum credit card, which is 100 USD.
2. After deducting the fee, the amount to be converted is 4900 USD.
3. The exchange rate from USD to EUR is 0.93.
4. The final amount is 4900 * 0.93 = 4557 EUR.


My agent now uses custom business logic with structured responses.